In [ ]:
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

Imports

In [ ]:
import pandas as pd
import numpy as np

# Tensorflow
import tensorflow as tf
import tensorflow_federated as tff

# Asynchronous, needed for federated learning
import nest_asyncio
nest_asyncio.apply()

# Tensorflow Keras
from keras.models import Sequential
from keras import regularizers
from keras.layers import (InputLayer, Dense, Flatten, Conv1D, MaxPool1D, Dropout, Activation)

# Others
import random
from sklearn.preprocessing import MinMaxScaler

#from google.colab import drive
#drive.mount('/gdrive', force_remount=True)

In [ ]:
def readAllDatasetsPaths(n_files_adl, n_files_fall):
  paths_fall = {}
  paths_adl = {}
  i=1
  for x in range (1, n_files_adl+1):
    p = "/Datasets/"+str(x)+".csv"
    paths_adl['client'+str(i)] = p
    i+=1
  for y in range(1, n_files_fall+1):
    p = "/Datasets/"+str(y)+".csv"
    paths_fall['client'+str(i)] = p
    i+=1
  return paths_adl, paths_fall

paths_adl, paths_fall = readAllDatasetsPaths(10, 10)

print(paths_adl)
print(paths_fall)

In [4]:
NUM_EPOCHS = 8
BATCH_SIZE = 16
SHUFFLE_BUFFER = 360
PREFETCH_BUFFER = 8
client_instance = 360

def load_client_ds(path_adl, path_fall, client_instance):

    h_client_instance = int(client_instance/2)

    #Defining one hot encode y
    y_adl = np.full((h_client_instance, 2), [1, 0])
    y_fall = np.full((h_client_instance, 2), [0, 1])
    y = np.concatenate((y_adl, y_fall), axis=0)

    #Reading datasets
    df_adl = pd.read_csv(path_adl, header=None)
    df_fall = pd.read_csv(path_fall, header=None)

    # Getting only accelerometer columns (x,y,z)
    X_adl = df_adl.iloc[:, :-1]
    X_fall = df_fall.iloc[:, :-1]

    #applying normalization
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_adl = scaler.fit_transform(X_adl)
    X_fall = scaler.fit_transform(X_fall)

    n1 = int(len(X_adl)/2)
    n2 = int(len(X_fall)/2)

    #Adding two classes a each dataset
    X1 = np.concatenate((X_adl[0:n1], X_fall[0:n2]), axis=0)
    X2 = np.concatenate((X_adl[n1:],  X_fall[n2:]), axis=0)

    X1 = np.reshape(X1, (client_instance, 500, 3))
    X2 = np.reshape(X2, (client_instance, 500, 3))

    # create tensorflow dataset
    dataset1 = tf.data.Dataset.from_tensor_slices((X1, y))
    dataset2 = tf.data.Dataset.from_tensor_slices((X2, y))

    dataset1 = dataset1.batch(BATCH_SIZE).repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).prefetch(PREFETCH_BUFFER)
    dataset2 = dataset2.batch(BATCH_SIZE).repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).prefetch(PREFETCH_BUFFER)

    return dataset1, dataset2

In [5]:
def get_dataset_partitions(dataset_list, dataset_list_len, train_split=0.8, test_split=0.2, shuffle=True):

  if shuffle:
   random.shuffle(dataset_list)

  n_train = int(dataset_list_len * train_split)

  train_data = dataset_list[:n_train]
  test_data = dataset_list[n_train:]

  return train_data, test_data

In [6]:
ds1, ds2 = map(list, zip(*[load_client_ds(paths_adl[x], paths_fall[y], client_instance) for (x, y) in zip(paths_adl, paths_fall)]))
clients_ds = ds1 + ds2

In [ ]:
train_data, test_data = get_dataset_partitions(clients_ds, len(clients_ds))
tf.data.experimental.cardinality(train_data[0])

Creating Keras Model

In [8]:
def create_keras_model():
    model = Sequential()

    model.add(InputLayer(input_shape=(500, 3)))
    n_classes = 2

    model.add(tf.keras.layers.Conv1D(32, 3, padding='causal', activation="relu"))
    model.add(tf.keras.layers.MaxPooling1D(2, strides=2))

    model.add(tf.keras.layers.Conv1D(64, 3, padding='causal', activation="relu"))
    model.add(tf.keras.layers.MaxPooling1D(2, strides=2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation="relu"))
    model.add(tf.keras.layers.Dense(n_classes, activation="softmax"))

    return model

**Creating FL model**

In [9]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec= train_data[0].element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc'),\
               tf.keras.metrics.Precision(name='pr'),\
               tf.keras.metrics.Recall(name='rc')])

In [10]:
model = create_keras_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 500, 32)           320       
                                                                 
 max_pooling1d (MaxPooling1  (None, 250, 32)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 250, 64)           6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 125, 64)           0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 8000)              0         
                                                                 
 dense (Dense)               (None, 100)               8

Federated Learning setup

In [11]:
NUM_ROUNDS = 5

client_lr = 1e-2
server_lr = 1e-2


# Simulate federated learning with federated averaging as model aggregation
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn= lambda: tf.keras.optimizers.SGD(client_lr),
    server_optimizer_fn= lambda: tf.keras.optimizers.SGD(server_lr))

# Init
train_state = training_process.initialize()

for i in range(NUM_ROUNDS):
    result = training_process.next(train_state, train_data)
    train_state = result.state
    metrics = result.metrics
    print('round {:5d}, metrics={}'.format(i, metrics))

round     0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('acc', 0.69134116), ('pr', 0.69134116), ('rc', 0.69134116), ('loss', 0.56044203), ('num_examples', 46080), ('num_batches', 2944)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round     1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('acc', 0.7019314), ('pr', 0.7019314), ('rc', 0.7019314), ('loss', 0.5501223), ('num_examples', 46080), ('num_batches', 2944)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round     2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('acc', 0.70360243), ('pr', 0.70360243), ('rc', 0.70360243), ('loss', 0.5433214), ('num_examples', 46080), ('num_batches', 2944)]))])), ('aggregator', 

In [12]:
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)
evaluation_state = evaluation_process.initialize() #item novo
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [13]:
training_output = evaluation_process.next(evaluation_state, train_data)
training_output.metrics

OrderedDict([('distributor', ()),
             ('client_work',
              OrderedDict([('eval',
                            OrderedDict([('current_round_metrics',
                                          OrderedDict([('acc', 0.5),
                                                       ('pr', 0.5),
                                                       ('rc', 0.5),
                                                       ('loss', 0.67714024),
                                                       ('num_examples', 46080),
                                                       ('num_batches',
                                                        2944)])),
                                         ('total_rounds_metrics',
                                          OrderedDict([('acc', 0.5),
                                                       ('pr', 0.5),
                                                       ('rc', 0.5),
                                                       ('loss', 

In [14]:
evaluation_output = evaluation_process.next(evaluation_state, test_data)
evaluation_output.metrics

OrderedDict([('distributor', ()),
             ('client_work',
              OrderedDict([('eval',
                            OrderedDict([('current_round_metrics',
                                          OrderedDict([('acc', 0.5),
                                                       ('pr', 0.5),
                                                       ('rc', 0.5),
                                                       ('loss', 0.6769778),
                                                       ('num_examples', 11520),
                                                       ('num_batches', 736)])),
                                         ('total_rounds_metrics',
                                          OrderedDict([('acc', 0.5),
                                                       ('pr', 0.5),
                                                       ('rc', 0.5),
                                                       ('loss', 0.6769778),
                                              